In [2]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 48.2 MB/s eta 0:00:00


In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [36]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def show_hand(video_path, interval):
  xyz_list_list , xyz_list_list_flip = [], []
  # cv2.destroyAllWindows() #열려있는 창 닫기
  hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.3)

  cap = cv2.VideoCapture(video_path)
  if cap.isOpened():
    cnt = 0
    while True:
      ret, img = cap.read()
      if cnt == interval and ret == True:
        cnt = 0
        xyz_list, xyz_list_flip = [], []
        img = cv2.resize(img, (640,640))
        results = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if not results.multi_hand_landmarks:
          continue

        # idx = 0
        # draw_line_dic = {}

        for x_y_z in results.multi_hand_landmarks:
          for landmark in x_y_z.landmark:
            xyz_list.append(landmark.x)
            xyz_list.append(landmark.y)
            xyz_list.append(landmark.z)
            xyz_list_flip.append(1 - landmark.x)
            xyz_list_flip.append(landmark.y)
            xyz_list_flip.append(landmark.z)
          # print(x_y_z)
          # xyz_list.append(x_y_z[x])
          # xyz_list.append(x_y_z.y)
          # xyz_list.append(x_y_z.z)
          # xyz_list_flip.append(1- x_y_z.x)
          # xyz_list_flip.append(x_y_z.y)
          # xyz_list_flip.append(x_y_z.z)
          # x, y , z = int(x_y_z.x * 640), int(x_y_z.y * 640), int(x_y_z.z * 640)
          # draw_line_dic[idx] = [x, y, z]
          # idx += 1
          mp_drawing.draw_landmarks(img, x_y_z, mp_hands.HAND_CONNECTIONS)
        xyz_list_list.append(xyz_list)
        xyz_list_list_flip.append(xyz_list_flip)

       
        cv2.imshow('Hand Landmarks', img)
        cv2.waitKey(1)
      elif ret == False:
        break
      cnt += 1
  cap.release()
  cv2.destroyAllWindows()
  return xyz_list_list +  xyz_list_list_flip

In [35]:
import os
import random

video_path = './data/video/train_data'
video_name_list = os.listdir(video_path)
dataset = []
length = 20
interval = 1

for video_name in video_name_list:
  if 'release' in video_name: label = 0
  else: label = 1
  hand_data = show_hand('{}/{}'.format(video_path, video_name), interval)
  for idx in range(0, len(hand_data), int(length/2)):
    seq_list = hand_data[idx : idx + length]
    if len(seq_list) == length:
      dataset.append({'key': label, 'value' : seq_list})

random.shuffle(dataset)


x: 0.7215449810028076
y: 0.748115062713623
z: 1.252013106523009e-07

x: 0.6590174436569214
y: 0.6519243121147156
z: -0.09839914739131927

x: 0.59085613489151
y: 0.5826413631439209
z: -0.06272660195827484

x: 0.5180284380912781
y: 0.5610346794128418
z: 0.0037702638655900955

x: 0.478459894657135
y: 0.5381948351860046
z: 0.0901157557964325

x: 0.6520874500274658
y: 0.4195011258125305
z: 0.027719318866729736

x: 0.5916760563850403
y: 0.3050467371940613
z: 0.04035688936710358

x: 0.5293739438056946
y: 0.2782859802246094
z: 0.047597773373126984

x: 0.4843997657299042
y: 0.2905493974685669
z: 0.06201860308647156

x: 0.6166093945503235
y: 0.46927499771118164
z: 0.10155154019594193

x: 0.5668497085571289
y: 0.35780853033065796
z: 0.12665621936321259

x: 0.5122716426849365
y: 0.3261459171772003
z: 0.1164700984954834

x: 0.4863426983356476
y: 0.32549983263015747
z: 0.11391200870275497

x: 0.5879022479057312
y: 0.522457480430603
z: 0.15672938525676727

x: 0.5066583156585693
y: 0.4830509126186371


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1267: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
print(len(dataset))

In [ ]:
dataset